In [1]:
import os
import zarr
import random
import json
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from collections import defaultdict
import sys

warnings.filterwarnings("ignore")
sys.path.append("./src/")

from src.config import CFG
from src.dataloader import (
    read_zarr,
    read_info_json,
    scale_coordinates,
    create_dataset,
    create_segmentation_map,
    EziiDataset,
    drop_padding,
)
from src.network import UNet_2D, UNet_3D, aug
from src.utils import save_images
from src.metric import score, create_cls_pos, create_cls_pos_sikii, create_df
from metric import visualize_epoch_results

In [2]:
train_dataset = EziiDataset(
    exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train/static",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.train_zarr_types,
)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

for data in train_loader:
    break

[('TS_4', 'denoised'), ('TS_5_4', 'denoised'), ('TS_21', 'denoised'), ('TS_3', 'denoised'), ('TS_1', 'denoised'), ('TS_73_6', 'denoised'), ('TS_7', 'denoised'), ('TS_19', 'denoised'), ('TS_5', 'denoised'), ('TS_23', 'denoised'), ('TS_12', 'denoised'), ('TS_26', 'denoised'), ('TS_8', 'denoised'), ('TS_99_9', 'denoised'), ('TS_20', 'denoised'), ('TS_6_4', 'denoised'), ('TS_69_2', 'denoised'), ('TS_17', 'denoised'), ('TS_0', 'denoised'), ('TS_16', 'denoised'), ('TS_2', 'denoised'), ('TS_10', 'denoised'), ('TS_25', 'denoised'), ('TS_24', 'denoised'), ('TS_22', 'denoised'), ('TS_6', 'denoised'), ('TS_9', 'denoised'), ('TS_15', 'denoised'), ('TS_18', 'denoised'), ('TS_13', 'denoised'), ('TS_14', 'denoised'), ('TS_11', 'denoised')]


In [3]:
train_dataset = EziiDataset(
    exp_names=CFG.train_exp_names,
    base_dir="../../inputs/train/static",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.train_zarr_types,
)

valid_dataset = EziiDataset(
    exp_names=CFG.valid_exp_names,
    base_dir="../../inputs/train/static",
    particles_name=CFG.particles_name,
    resolution=CFG.resolution,
    zarr_type=CFG.valid_zarr_types,
)

from tqdm import tqdm

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
train_nshuffle_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False)

for data in tqdm(train_loader):
    normalized_tomogram = data["normalized_tomogram"].unsqueeze(0)
    segmentation_map = data["segmentation_map"].unsqueeze(0)
    break

# normalized_tomogram =
normalized_tomogram.shape, segmentation_map.shape

[('TS_4', 'denoised'), ('TS_5_4', 'denoised'), ('TS_21', 'denoised'), ('TS_3', 'denoised'), ('TS_1', 'denoised'), ('TS_73_6', 'denoised'), ('TS_7', 'denoised'), ('TS_19', 'denoised'), ('TS_5', 'denoised'), ('TS_23', 'denoised'), ('TS_12', 'denoised'), ('TS_26', 'denoised'), ('TS_8', 'denoised'), ('TS_99_9', 'denoised'), ('TS_20', 'denoised'), ('TS_6_4', 'denoised'), ('TS_69_2', 'denoised'), ('TS_17', 'denoised'), ('TS_0', 'denoised'), ('TS_16', 'denoised'), ('TS_2', 'denoised'), ('TS_10', 'denoised'), ('TS_25', 'denoised'), ('TS_24', 'denoised'), ('TS_22', 'denoised'), ('TS_6', 'denoised'), ('TS_9', 'denoised'), ('TS_15', 'denoised'), ('TS_18', 'denoised'), ('TS_13', 'denoised'), ('TS_14', 'denoised'), ('TS_11', 'denoised')]
[('TS_86_3', 'denoised'), ('TS_6_6', 'denoised')]


  0%|          | 0/32 [00:01<?, ?it/s]


(torch.Size([1, 1, 50, 160, 160]), torch.Size([1, 1, 50, 160, 160]))

In [4]:
import torch

model = UNet_3D()
test_input = torch.randn(1, 1, 46, 160, 160)

model(test_input).shape

torch.Size([1, 7, 46, 160, 160])

In [5]:
import torch
import torchvision.transforms.functional as F
import random

In [6]:
model = UNet_3D().to("cuda")
model.eval()


optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3)
criterion = nn.CrossEntropyLoss(
    # weight=torch.tensor([0, 0.1000, 14.4163, 14.1303, 1.0000, 2.2055, 4.4967]).to(
    #     "cuda"
    # )
    weight=torch.tensor([0.5, 32, 32, 32, 32, 32, 32]).to("cuda")
)
# criterion = DiceLoss()

best_model = None
best_score = 0

for epoch in range(100):
    train_loss = []
    valid_loss = []
    train_pred_tomogram = defaultdict(list)
    train_gt_tomogram = defaultdict(list)
    valid_pred_tomogram = defaultdict(list)
    valid_gt_tomogram = defaultdict(list)
    model.train()
    tq = tqdm(range(len(train_loader)))
    for data in train_loader:
        exp_name = data["exp_name"][0]
        tomogram = data["normalized_tomogram"].unsqueeze(0)
        segmentation_map = data["segmentation_map"].long()  # .unsqueeze(0)

        optimizer.zero_grad()
        input_ = tomogram
        # input_, gt = aug(input_, gt)
        input_ = input_.to("cuda")
        gt = segmentation_map.to("cuda")
        output = model(input_)
        loss = criterion(output, gt)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        tq.set_description(f"Train-Epoch: {epoch}, Loss: {np.mean(train_loss)}")
        tq.update(1)

        train_pred_tomogram[exp_name].append(output.squeeze(0).cpu().detach().numpy())
        train_gt_tomogram[exp_name].append(gt.squeeze(0).cpu().detach().numpy())

    tq.close()

    train_score_ = visualize_epoch_results(
        train_pred_tomogram,
        train_gt_tomogram,
        sikii_dict=CFG.initial_sikii,
    )

    print(f"EPOCH: {epoch}, TRAIN_SCORE: {train_score_}")

    # # 可視化
    # index = 20

    # plt.figure(figsize=(10, 5))

    # ax = plt.subplot(1, 4, 1)
    # ax.imshow(train_pred_tomogram[exp_name][index].squeeze(0).argmax(0))
    # ax.set_title("Train-Prediction")
    # ax.axis("off")

    # ax = plt.subplot(1, 4, 2)
    # ax.imshow(train_gt_tomogram[exp_name][index].squeeze(0))
    # ax.set_title("Train-Ground Truth")
    # ax.axis("off")

    ############################################# valid #############################################

    model.eval()
    tq = tqdm(range(len(valid_loader)))
    for data in valid_loader:
        exp_name = data["exp_name"][0]
        tomogram = data["normalized_tomogram"].unsqueeze(0)
        segmentation_map = data["segmentation_map"].to("cuda").long()

        input_ = tomogram
        # input_, gt = aug(input_, gt)
        input_ = input_.to("cuda")
        gt = segmentation_map.to("cuda")
        output = model(input_)
        loss = criterion(output, gt)
        loss.backward()
        optimizer.step()

        valid_loss.append(loss.item())
        tq.set_description(f"Valid-Epoch: {epoch}, Loss: {np.mean(valid_loss)}")
        tq.update(1)

        valid_pred_tomogram[exp_name].append(output.squeeze(0).cpu().detach().numpy())
        valid_gt_tomogram[exp_name].append(gt.squeeze(0).cpu().detach().numpy())
    tq.close()

    valid_score_ = visualize_epoch_results(
        valid_pred_tomogram,
        valid_gt_tomogram,
        sikii_dict=CFG.initial_sikii,
    )

    print(f"EPOCH: {epoch}, VALID_SCORE: {valid_score_}")

    if valid_score_ > best_score:
        best_score = valid_score_
        best_model = model
        torch.save(model.state_dict(), f"best_model.pth")

    # ax = plt.subplot(1, 4, 3)
    # ax.imshow(valid_pred_tomogram[exp_name][index].argmax(1).squeeze(0))
    # ax.set_title("Valid-Prediction")
    # ax.axis("off")

    # ax = plt.subplot(1, 4, 4)
    # ax.imshow(valid_gt_tomogram[exp_name][index].squeeze(0))
    # ax.set_title("Valid-Ground Truth")
    # ax.axis("off")

    # plt.tight_layout()

    # plt.show()

    # save_images(
    #     train_gt_tomogram=train_gt_tomogram,
    #     train_pred_tomogram=train_pred_tomogram,
    #     valid_gt_tomogram=valid_gt_tomogram,
    #     valid_pred_tomogram=valid_pred_tomogram,
    #     save_dir="images",
    #     epoch=epoch,
    # )

Train-Epoch: 0, Loss: 3.026161205023527: 100%|██████████| 32/32 [00:54<00:00,  1.72s/it] 


EPOCH: 0, TRAIN_SCORE: 0.01792845311861243


Valid-Epoch: 0, Loss: 1.673833191394806: 100%|██████████| 2/2 [00:01<00:00,  1.64it/s]


EPOCH: 0, VALID_SCORE: 0.04001366690710299


Train-Epoch: 1, Loss: 1.101269906386733: 100%|██████████| 32/32 [00:53<00:00,  1.68s/it] 


...

EPOCH: 98, VALID_SCORE: 0.04001366690710299


Train-Epoch: 99, Loss: 0.6406160891056061: 100%|██████████| 32/32 [00:52<00:00,  1.63s/it]


EPOCH: 99, TRAIN_SCORE: 0.05225635744124932


Valid-Epoch: 99, Loss: 1.3289843797683716: 100%|██████████| 2/2 [00:01<00:00,  1.61it/s]


EPOCH: 99, VALID_SCORE: 0.04001366690710299
